# Проект № 12.3: Проект по SQL

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Основные задачи проекта

**Задания:**

* посчитать, сколько книг вышло после 1 января 2000 года;
* для каждой книги посчитать количество обзоров и среднюю оценку;
* определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
* определить автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;
* посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Описание данных

**Таблица `books` (cодержит данные о книгах):**

* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

**Таблица `authors` (содержит данные об авторах):**

* `author_id` — идентификатор автора;
* `author` — имя автора.

**Таблица `publishers` (содержит данные об издательствах):**

* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства.

**Таблица `ratings` (содержит данные о пользовательских оценках книг):**

* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

**Таблица `reviews` (содержит данные о пользовательских обзорах):**

* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.

### Схема данных

![Схема данных](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

## Исследование общей информации о таблицах

### Импорт библиотек, подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

Подключимся к базе данных:

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Общая информация о таблицах

In [3]:
# с помощью цикла выведем первые 5 строк таблицы и общую информацию
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = ''' SELECT* 
                FROM {}
            '''.format(table) 
    request = pd.io.sql.read_sql(query, con = engine) 
    display(request.head())
    display(request.info())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

Пропущенных значений в таблицах нет.

## Основная часть проекта

In [4]:
# функция для выполнения SQL-запроса с помощью pandas
def query_sql(sql):
    return pd.io.sql.read_sql(sql, con = engine)

### Задание № 1. Посчитать, сколько книг вышло после 1 января 2000 года

In [5]:
# посчитаем, сколько книг вышло после 1 января 2000 года
query_1 = ''' 
SELECT COUNT(DISTINCT book_id) as books_count
FROM books
WHERE publication_date > '2000-1-1'; 
''' 

# выполним SQL-запрос с помощью pandas
query_sql(query_1)

,books_count
0,819


После 1 января 2000 года было выпущено 819 книг.

### Задание № 2. Для каждой книги посчитать количество обзоров и среднюю оценку

In [6]:
# посчитаем для каждой книги количество обзоров и среднюю оценку
query_2 = ''' 
SELECT 
    b.book_id AS book_id,
    b.title AS book_name,
    rv.count AS reviews_count,
    ROUND (rt.avg_rat, 1) AS avg_rating
FROM books AS b 
LEFT JOIN 
         (SELECT book_id,
                 AVG(rating) AS avg_rat
          FROM ratings
          GROUP BY book_id
         ) AS rt ON b.book_id = rt.book_id
LEFT JOIN
         (SELECT book_id,
                 COUNT(review_id) AS count
          FROM reviews
          GROUP BY book_id
          ) AS rv ON b.book_id = rv.book_id; 
''' 

# выполним SQL-запрос с помощью pandas
query_sql(query_2)

,book_id,book_name,reviews_count,avg_rating
0,652,The Body in the Library (Miss Marple #3),2.0,4.5
1,273,Galápagos,2.0,4.5
2,51,A Tree Grows in Brooklyn,5.0,4.3
3,951,Undaunted Courage: The Pioneering First Missio...,2.0,4.0
4,839,The Prophet,4.0,4.3
...,...,...,...,...
995,64,Alice in Wonderland,4.0,4.2
996,55,A Woman of Substance (Emma Harte Saga #1),2.0,5.0
997,148,Christine,3.0,3.4
998,790,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.5


Для каждой книги были добавлены столбца с количеством обзоров и средней оценкой.

### Задача № 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.

In [7]:
# определим издательство, которое выпустило наибольшее число книг толще 50 страниц
query_3 = ''' 
SELECT p.publisher_id,
       p.publisher,
       COUNT(b.book_id) AS books_count
FROM publishers AS p
INNER JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 1;
''' 

# выполним SQL-запрос с помощью pandas
query_sql(query_3)

,publisher_id,publisher,books_count
0,212,Penguin Books,42


Издательство **«Penguin Books»** выпустило наибольшее число книг толще 50 страниц - 42.

### Задача № 4. Определить автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;

In [8]:
# определим автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)
query_4 = ''' 
SELECT a.author_id,
       a.author,
       ROUND(AVG(rt.rating), 1) AS avg_rating
FROM authors AS a
INNER JOIN books AS b ON a.author_id = b.author_id
INNER JOIN ratings AS rt ON b.book_id = rt.book_id
WHERE rt.book_id IN (SELECT book_id
                      FROM ratings
                      GROUP BY book_id
                      HAVING COUNT(rating_id) >= 50
                      )
GROUP BY a.author_id
ORDER BY avg_rating DESC
LIMIT 1;
'''            

# выполним SQL-запрос с помощью pandas
query_sql(query_4)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.3


Самая высокая средняя оценка книг (4.3) у Mary GrandPré - американский иллюстратор, наиболее известный своими иллюстрациями обложек и глав книг о Гарри Поттере и J.K. Rowling - писательница, которая и придумала Гарри Поттера.

### Задача № 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [9]:
# посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок
query_5 = ''' 
SELECT AVG(rv.rev_count) AS avg_review_count
FROM
    (SELECT username,
            COUNT(review_id) AS rev_count
     FROM reviews
     WHERE username IN (SELECT username
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating_id) > 50
                       )
     GROUP BY username
    ) AS rv; 
''' 

# выполним SQL-запрос с помощью pandas
query_sql(query_5)

,avg_review_count
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - чуть больше 24.

## Вывод

Наша задача как аналитика — проанализировать базу данных. Перед нами стояло 5 задач, для решения которых мы выполнили подключение к базе данных сервиса для чтения книг, изучили общую информацию о таблицах и с помощью SQL-запросов получили необходимую информацию. Эти данные помогут сформулировать ценностное предложение для нового продукта.

1. После 1 января 2000 года было выпущего 819 книг.
2. Для каждой книги посчитали количество обзоров и их среднюю оценку.
3. Издательство «Penguin Books» выпустило наибольшее число книг толще 50 страниц - 42.
4. Самая высокая средняя оценка книг (4.3) у Mary GrandPré - американский иллюстратор, наиболее известный своими иллюстрациями обложек и глав книг о Гарри Поттере и J.K. Rowling - писательница, которая и придумала Гарри Поттера.
5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - чуть больше 24.